# Connect4-Zero Training on Colab

AlphaZero-style training for Connect 4. Make sure to use a GPU runtime:
- Runtime > Change runtime type > GPU (T4 is fine, A100 is faster)

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repo
!git clone https://github.com/mindswim/connect4-zero.git
%cd connect4-zero

In [ ]:
# Install dependencies
!pip install -e . -q

In [ ]:
# Verify CUDA is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# Quick benchmark
!c4z benchmark --sims 100 --games 5

In [ ]:
# Train! This will take a while but show progress
# Use --iterations to control how long
!c4z train --iterations 50

In [ ]:
# Save checkpoint to Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

!cp -r checkpoints /content/drive/MyDrive/connect4-zero-checkpoints

In [ ]:
# Test against random player
from connect4zero.net import load_checkpoint
from connect4zero.eval import Arena
from connect4zero.utils import get_device

device = get_device()
model, _ = load_checkpoint('checkpoints/best.pt', device)

arena = Arena(num_simulations=100, device=device)
result = arena.evaluate_vs_random(model, num_games=50)
print(f"Win rate vs random: {result.win_rate*100:.1f}%")

In [ ]:
# Download best model
from google.colab import files
files.download('checkpoints/best.pt')